# EMOD Pseudocode

_Indented_ code under commented out functions is the pseudocode for the function. E.g., `updateInfectivity(dt)` calls `computeMaxInfectionProb(dt)` and `gap = calcGap()`.

```cpp
void Node::Update(float dt) {
    ...
    // updateInfectivity(dt);
        // computeMaxInfectionProb( dt );
            ProbabilityNumber prob = EXPCDF(-contagion * dt);
            gap = 1;           // default to visiting every individual
            bSkipping = false; // default to visiting (!skipping) every individual
        // gap = calcGap();
            int gap = 1;
            float maxProb = prob;
            if (maxProb >= 1.0) {
                gap = 1;
            }
            else if (maxProb > 0.0) {
                // geometric based on maxProb
                gap = int(ceil(log(1.0 - GetRng()->e()) / log(1.0 - maxProb)));	

    ...
    for (int i = 0; i < individualHumans.size(); ++i ) {
    	// PreUpdate();
            if (gap == 1) {
                bSkipping = false;  // visit this individual
                gap = calcGap();    // update gap (see above)
            }
            else {
                bSkipping = true;   // skip this individual
                --gap;              // decrement gap counter
            }
        // individualHumans[i]->Update(...);
            // ExposeToInfectivity(...);
                // parent->ExposeIndividual(...);
                    if (!bSkipping) {
                        // transmissionGroups->ExposeToContagion(...);
                            // candidate->Expose(...);
                                auto indProb = -cp->GetTotalContagion() * dt;
                                indProb *= susceptibility->getModAcquire();
                                indProb *= susceptibility->getModRisk();
                                indProb *= interventions->GetInterventionReducedAcquire(...);
                                indProb = EXPCDF(indProb);
                                bool acquire = false;
                                float maxProb = parent->GetMaxInfectionProb(...);
                                if (maxProb > 0) {
                                    // if individual is maximally susceptible
                                    if (maxProb == prob) {
                                        acquire = true;
                                    }
                                    // if individual is _not_ maximally susceptible
                                    else if (prob/maxProb > GetRng()->e()) {
                                        acquire = true;
                                    }
                                }
                    }
    }
    ...
}

#define EXPCDF(x)   (1 - exp(x))
```

In [ ]:
%pip install numba

In [1]:
import numpy as np
import numba as nb

In [3]:
@nb.njit()
def skip(probability: np.float32) -> np.uint32:
    n = np.uint32(np.ceil(np.log(1.0-np.random.rand())/np.log(1.0-probability))) if probability < 1.0 else np.uint32(1)
    return n

In [27]:
@nb.njit()
def visit(susceptibility: np.ndarray, probability: np.float32) -> np.uint32:
    visited = 0
    index = skip(probability) - 1
    while (index >= 0) and (index < len(susceptibility)):
        if susceptibility[index] == 1.0:
            visited += 1
        elif np.random.rand() < susceptibility[index]:
            visited += 1
        index += skip(probability)
    return visited

In [64]:
susceptibility = np.ones(1_000_000, dtype=np.float32)

In [65]:
# randomize susceptibility uniformly between 0 and 1
susceptibility *= np.random.uniform(0.0, 1.0, size=susceptibility.shape)

In [ ]:
susceptibility[0:32]

array([0.99719566, 0.8540871 , 0.10468864, 0.35516334, 0.42941552,
       0.7124959 , 0.58953863, 0.72594905, 0.4248593 , 0.9042252 ,
       0.56570256, 0.6228677 , 0.11523604, 0.13943467, 0.02241728,
       0.32916895, 0.95228964, 0.6738025 , 0.77935255, 0.1789487 ,
       0.01528036, 0.23065953, 0.00682879, 0.58484095, 0.18699397,
       0.5571306 , 0.91140044, 0.9502877 , 0.5744521 , 0.34150472,
       0.23320895, 0.54531837], dtype=float32)

In [66]:
for contagion in [1.0, 2.0, 4.0, 8.0, 16.0, 32.0, 64.0, 128.0, 256.0]:
    probability = contagion / susceptibility.shape[0]
    tries = 100_000
    visits = np.zeros(tries, dtype=np.uint32)
    for i in range(tries):
        visits[i] = visit(susceptibility, probability)
    print(f"Contagion: {contagion:>5.0f}, Probability: {probability:>6f}, Visits: {visits.mean():>6.2f} ± {visits.std():5.2f} ", end="")
    print(f"mean sus.: {susceptibility.mean():3.2}, eff. prob.: {probability*(susceptibility.mean()):>6f}")

Contagion:     1, Probability: 0.000001, Visits:   0.50 ±  0.71 mean sus.: 0.5, eff. prob.: 0.000001
Contagion:     2, Probability: 0.000002, Visits:   1.00 ±  1.00 mean sus.: 0.5, eff. prob.: 0.000001
Contagion:     4, Probability: 0.000004, Visits:   2.01 ±  1.42 mean sus.: 0.5, eff. prob.: 0.000002
Contagion:     8, Probability: 0.000008, Visits:   4.01 ±  2.00 mean sus.: 0.5, eff. prob.: 0.000004
Contagion:    16, Probability: 0.000016, Visits:   8.01 ±  2.84 mean sus.: 0.5, eff. prob.: 0.000008
Contagion:    32, Probability: 0.000032, Visits:  15.99 ±  4.00 mean sus.: 0.5, eff. prob.: 0.000016
Contagion:    64, Probability: 0.000064, Visits:  32.04 ±  5.65 mean sus.: 0.5, eff. prob.: 0.000032
Contagion:   128, Probability: 0.000128, Visits:  64.04 ±  8.02 mean sus.: 0.5, eff. prob.: 0.000064
Contagion:   256, Probability: 0.000256, Visits: 128.07 ± 11.34 mean sus.: 0.5, eff. prob.: 0.000128
